In [ ]:
import pandas as pd
from datetime import datetime as dt
from copy import deepcopy
from itertools import product

data is taken from kaggle: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?select=olist_orders_dataset.csv

files in this dataset:
*   olist_customers_dataset.csv	  
*   olist_orders_dataset.csv
*   olist_geolocation_dataset.csv	  
*   olist_products_dataset.csv
*   olist_order_items_dataset.csv	  
*   olist_sellers_dataset.csv
*   olist_order_payments_dataset.csv  
*   product_category_name_translation.csv
*   olist_order_reviews_dataset.csv


In [ ]:
# product info
products = pd.read_csv(f'{base_dir}/olist_products_dataset.csv', usecols=['product_id', 'product_category_name'])
translation = pd.read_csv(f'{base_dir}/product_category_name_translation.csv')

# original category name column is in portuguese, so we join on another
# dataframe that has category names in english.
products = products.merge(translation, how='left', on='product_category_name')

print(products[['product_category_name_english']].value_counts())

In [ ]:
# order info
orders = pd.read_csv(f'{base_dir}/olist_orders_dataset.csv', usecols=['order_id', 'order_purchase_timestamp'])
order_items = pd.read_csv(f'{base_dir}/olist_order_items_dataset.csv', usecols=['order_id', 'product_id', 'price'])

# joining on order_id
orders = orders.merge(order_items, how='left', on='order_id')

# adding an order date column
orders['order_date'] = orders.order_purchase_timestamp.apply(lambda timestamp: dt.strptime(timestamp, '%Y-%m-%d %H:%M:%S').date())

# filtering for 2017 onwards (there weren't as many orders in 2016) and sorting by date
orders = orders[orders.order_date >= dt.fromisoformat('2017-01-01').date()].sort_values('order_date')

In [ ]:
# joining on products
orders = orders.merge(products, how='inner', on='product_id')

orders['order_week'] = orders.order_date.apply(lambda date: date.strftime('%Y-%W'))
agg_orders = orders.groupby(['product_category_name_english', 'order_week']).size().reset_index(name='orders').sort_values('order_week')
largest_categories = agg_orders.groupby('product_category_name_english', as_index=False).sum(numeric_only=True)[['product_category_name_english', 'orders']].sort_values('orders', ascending=False)['product_category_name_english'].tolist()[:5]
df = deepcopy(agg_orders[agg_orders.product_category_name_english.isin(largest_categories)])
df['category'] = df['product_category_name_english']
df = df[['category', 'order_week', 'orders']]

In [ ]:
# all unique week and category combinations over the date range
date_range = pd.date_range(min(orders.order_date), max(orders.order_date))
weeks = pd.unique([date.strftime('%Y-%W') for date in date_range]).tolist()
combinations = list(product(weeks, largest_categories))
# making a dataframe from the combinations
fill_df = pd.DataFrame({'order_week': [week for (week, category) in combinations], 'category': [category for (week, category) in combinations]})
# joining on old df for nonzero values and then filling nulls as having 0 orders
fill_df = fill_df.merge(df, how='left', on=('category', 'order_week')).fillna(0)
df = deepcopy(fill_df[['category', 'order_week', 'orders']])
df['orders'] = df['orders'].astype(int)

In [ ]:
df.sort_values('order_week')

In [ ]:
df.to_csv(f'./cleaned_orders.csv', index=False)